In [8]:
import json
import subprocess
import papermill as pm
import os
from datetime import datetime
import tempfile
import nbformat
import stat

In [9]:
# Definir el directorio base y temporal
try:
    BASE_DIR = os.path.abspath(os.path.dirname(__file__))
except NameError:
    BASE_DIR = os.getcwd()

TEMP_DIR = os.path.join(BASE_DIR, 'temp')
os.makedirs(TEMP_DIR, exist_ok=True)
os.chmod(TEMP_DIR, 0o777)


In [10]:
# Función para ejecutar notebooks
def ejecutar_notebook(notebook_path, parameters=None):
    if parameters is None:
        parameters = {}
    tmp_output_path = os.path.join(TEMP_DIR, 'output_notebook.ipynb')
    pm.execute_notebook(
        notebook_path,
        tmp_output_path,  # Usar un archivo temporal para el output
        parameters=parameters
    )

In [11]:
COMMON_NOTEBOOK_PATH = os.path.join(BASE_DIR, "01_LecturayAnalisis.ipynb")

In [12]:
# Ejecutar el notebook común
ejecutar_notebook(COMMON_NOTEBOOK_PATH)

c:\Users\Administrador\Documents\Python Scripts\Tesis\tesisaustral\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Executing: 100%|██████████| 14/14 [00:51<00:00,  3.69s/cell]


In [13]:
# Variantes de normalización
variantes = ['No_Norm', 'MinMax', 'Std', 'Maxabs', 'Robust']

In [14]:
# Ejecutar notebooks específicos para cada variante del experimento
for variante in variantes:
    notebook_path = os.path.join(BASE_DIR, f"02_Norm_{variante}.ipynb")
    #output_notebook = os.path.join(OUTPUT_DIR, f"02_Norm_{variante}_{datetime.now().strftime('%Y%m%d')}.ipynb")
    ejecutar_notebook(notebook_path, parameters={})
    
print("Ejecución de notebooks de normalización completada.")

Executing: 100%|██████████| 5/5 [00:21<00:00,  4.38s/cell]

Ejecución de notebooks de normalización completada.


In [15]:
for variante in variantes:
    notebook_path = os.path.join(BASE_DIR, f"03_Train_Test.ipynb")  # Usar el nombre original del notebook
    parameters = {
        'variante': variante,
        'test_size': 0.2,
        'random_state': 42  
    }
    ejecutar_notebook(notebook_path, parameters)

print("Ejecución de notebooks de normalización completada.")

Executing:   0%|          | 0/9 [00:00<?, ?cell/s]

Executing: 100%|██████████| 9/9 [00:25<00:00,  2.88s/cell]

Ejecución de notebooks de normalización completada.


In [16]:
#Parámetros TPOT
tpot_config = {
    "generations": 5,
    "population_size": 20,
    "verbosity": 2
}

In [17]:
# Crear un diccionario de configuración
config = {
    "data_paths": {
        # Asegúrate de que cada variante tenga sus correspondientes rutas definidas
        "X_train_No_Norm": "X_train_No_Norm.parquet",
        "X_test_No_Norm": "X_test_No_Norm.parquet",
        "y_train_No_Norm": "y_train_No_Norm.parquet",
        "y_test_No_Norm": "y_test_No_Norm.parquet",
        "X_train_MinMax": "X_train_MinMax.parquet",
        "X_test_MinMax": "X_test_MinMax.parquet",
        "y_train_MinMax": "y_train_MinMax.parquet",
        "y_test_MinMax": "y_test_MinMax.parquet",
        "X_train_Std": "X_train_Std.parquet",
        "X_test_Std": "X_test_Std.parquet",
        "y_train_Std": "y_train_Std.parquet",
        "y_test_Std": "y_test_Std.parquet",
        "X_train_Maxabs": "X_train_Maxabs.parquet",
        "X_test_Maxabs": "X_test_Maxabs.parquet",
        "y_train_Maxabs": "y_train_Maxabs.parquet",
        "y_test_Maxabs": "y_test_Maxabs.parquet",
        "X_train_Robust": "X_train_Robust.parquet",
        "X_test_Robust": "X_test_Robust.parquet",
        "y_train_Robust": "y_train_Robust.parquet",
        "y_test_Robust": "y_test_Robust.parquet"
    },
    "tpot_config": tpot_config
}

In [18]:
config_path = os.path.join(BASE_DIR, 'control_config.json')
with open(config_path, 'w') as f:
    json.dump(config, f)

In [ ]:
# Ejecutar el script de modelos
subprocess.run(['python', os.path.join(BASE_DIR, '04_Auto_Models.py')])